In [270]:
import subprocess
import os
import json
from structure import *
import re
from tqdm import tqdm
import operator
import time
from gensim import corpora, models
import numpy as np
from sklearn import neighbors

In [ ]:
os.listdir('D:\\proger\\test\\txts')[5000:]

Переименовываю контракты в contractN.txt:

In [7]:
i = 0
for filename in tqdm(os.listdir('D:\\proger\\test\\txts')):
    new_name = 'contract{}.txt'.format(i)
    os.rename('D:\\proger\\test\\txts\\' + filename, 'D:\\proger\\test\\txts\\' + new_name)
    i = i + 1

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5231.89it/s]


Сохраняю контракты в формате pickle:

In [ ]:
# txts - folder contained texts
# txts_pickled - folder contained pickled texts
for j in tqdm(range(10000)):
    d = Document('D:\\proger\\test\\txts\\contract{}.txt'.format(j))
    d.save('D:\\proger\\test\\txts_pickled\\', 'contract{}'.format(j))

Удаляю те документы, в которых не обнаружились главы:

In [ ]:
for filename in tqdm(os.listdir('D:\\proger\\test\\txts_pickled\\')):
    doc = Document()
    doc.load('D:\\proger\\test\\txts_pickled\\' + filename)
    if len(doc.get_titles_of_chapters()) == 0:
        os.remove('D:\\proger\\test\\txts_pickled\\' + filename)
        os.remove('D:\\proger\\test\\txts\\' + filename + '.txt')

Далее функции для обработки параграфов, которые делают следующее:
    regexp_tokenize - с помощью регулярных выражений оставляем только слова, состоящие из символов латинского алфавита и кириллицы, а также цифры (в которых мб '.' или ','), все цифры заменены на цифру 1.
    remove_stop_words - удаляем стоп-слова
    normalize 
    tokenize

In [7]:
def regexp_tokenize(sentence):
    pattern = re.compile(r'(\d+[.|,]*)+')
    tokenizer = RegexpTokenizer(r'[а-яА-ЯA-Za-z]+|1')
    return tokenizer.tokenize(pattern.sub('1', sentence))

#s = 'Товар поставляемый по настоящему договору покупается для заказа- заводской № 230, 231, строящихся по Государственным  Контрактам №78-ГК/11 от 02.12.2012 и  №80-ГК/11 от 02.12.12'
#print(s)
#rs = regexp_tokenize(s)
#pattern = re.compile(r'[\d.,]+')
#res = pattern.sub('1', s)
#print(rs)

In [8]:
def remove_stop_words(tokens):
    pattern = r'[^\w-]'
    stop_words = get_stop_words('russian')
    good_tokens = []
    for token in tokens:
        if not re.search(pattern, token) and token.lower() not in stop_words:
            good_tokens.append(token)
    return good_tokens

In [9]:
def normalize(tokens):
    morph = pymorphy2.MorphAnalyzer()
    norm_tokens = []
    for token in tokens:
        norm_tokens.append(morph.parse(token)[0].normal_form)
    return norm_tokens

In [10]:
def tokenize(sentence):
    """
    Get tokens of sentence. Uses `nltk.word_tokenize`.
    """
    return word_tokenize(sentence, language='russian')

Пример токенизации предложения

In [12]:
s = ['Товар поставляемый по настоящему договору покупается для заказа- заводской № 230, 231, строящихся по Государственным  Контрактам №78-ГК/11 время деньги от 02.12.2012  договор и  №80-ГК/11 от 02.12.12', 
     'Получение Покупателем оригиналов документов, указанных в статье 4.3. Договора;'];
res = regexp_tokenize(' '.join(s))
print('regexptokenize', res)
res = remove_stop_words(res)
print('removestopwords', res)
res = normalize(res)
print('norm', res)

s1 = 'Получение Покупателем оригиналов документов, указанных в статье 4.3. Договора;'
res = normalize(remove_stop_words(regexp_tokenize(s1)))
print(res)

regexptokenize ['Товар', 'поставляемый', 'по', 'настоящему', 'договору', 'покупается', 'для', 'заказа', 'заводской', '1', '1', 'строящихся', 'по', 'Государственным', 'Контрактам', '1', 'ГК', '1', 'время', 'деньги', 'от', '1', 'договор', 'и', '1', 'ГК', '1', 'от', '1', 'Получение', 'Покупателем', 'оригиналов', 'документов', 'указанных', 'в', 'статье', '1', 'Договора']
removestopwords ['Товар', 'поставляемый', 'настоящему', 'договору', 'покупается', 'заказа', 'заводской', '1', '1', 'строящихся', 'Государственным', 'Контрактам', '1', 'ГК', '1', 'деньги', '1', 'договор', '1', 'ГК', '1', '1', 'Получение', 'Покупателем', 'оригиналов', 'документов', 'указанных', 'статье', '1', 'Договора']
norm ['товар', 'поставлять', 'настоящий', 'договор', 'покупаться', 'заказ', 'заводский', '1', '1', 'строиться', 'государственный', 'контракт', '1', 'гк', '1', 'деньга', '1', 'договор', '1', 'гк', '1', '1', 'получение', 'покупатель', 'оригинал', 'документ', 'указанный', 'статья', '1', 'договор']
['получение',

In [50]:
points = []
is_correct = []
dictionary = dict()
for i, filename in tqdm(enumerate(os.listdir('D:\\proger\\test\\txts_pickled\\'))):
    points.append([])
    is_correct.append([])
    doc = Document()
    doc.load('D:\\proger\\test\\txts_pickled\\' + filename)

    # представляем весь договор в виде списка из последовательных пунктов этого договора:

    #  title
    sentences = [s.sentence for s in doc.title.sentences]
    tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(sentences))))
    points[i].append(tok_sent)
    is_correct[i].append(True)
    for word in tok_sent:
        if word in dictionary:
            dictionary[word] += 1 
        else:
            dictionary[word] = 1

    # preamble
    sentences = [s.sentence for s in doc.preamble.sentences]
    tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(sentences))))
    points[i].append(tok_sent)
    is_correct[i].append(True)
    for word in tok_sent:
        if word in dictionary:
            dictionary[word] += 1 
        else:
            dictionary[word] = 1    
    
    # numered text
    chapters = doc.get_titles_of_chapters()
    for chapter in chapters:
        doc[chapter].get_sentences()
        for paragraph in doc[chapter].queue:
            par_sent = paragraph.sentences
            pnts = [pnt.sentence for pnt in par_sent]
            tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(pnts))))
            points[i].append(tok_sent)
            is_correct[i].append(True)
            for word in tok_sent:
                if word in dictionary:
                    dictionary[word] += 1 
                else:
                    dictionary[word] = 1

    # attachment
    length = 0
    attachment = []
    for sentence in doc.attachment.sentences:
        tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(sentence.sentence))))
        if len(attachment) < 100:
            attachment.extend(tok_sent)
        else:
            points[i].append(attachment)
            is_correct[i].append(True)
            for word in attachment:
                if word in dictionary:
                    dictionary[word] += 1 
                else:
                    dictionary[word] = 1
            attachment = []
            #newPoint = Point(i, sentence_number, sentence.sentence, True)
            #db_session.add(newPoint)
            #sentence_number = sentence_number + 1
    #db_session.commit()
with open('points.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  1%|█                                                                          | 135/10000 [14:43<17:56:00,  6.54s/it]


  3%|██                                                                         | 271/10000 [32:44<19:35:41,  7.25s/it]


  4%|███                                                                        | 407/10000 [49:19<19:22:27,  7.27s/it]


  5%|███▉                                                                     | 543/10000 [1:06:22<19:16:05,  7.33s/it]


  7%|████▉                                                                    | 679/10000 [1:24:37<19:21:35,  7.48s/it]


...

 95%|████████████████████████████████████████████████████████████████████▌   | 9519/10000 [25:22:46<1:16:56,  9.60s/it]


 97%|███████████████████████████████████████████████████████████████████████▍  | 9655/10000 [25:42:06<55:06,  9.58s/it]


 98%|████████████████████████████████████████████████████████████████████████▍ | 9791/10000 [26:10:30<33:31,  9.62s/it]


 99%|█████████████████████████████████████████████████████████████████████████▍| 9927/10000 [26:40:12<11:46,  9.67s/it]


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [26:51:23<00:00,  9.67s/it]



Получение словаря коллекции с частотой встречемости каждого термина

In [50]:
with open('points.pickle', 'rb') as f:
    points = pickle.load(f)
dictionary = dict()
for document in tqdm(points):
    for par in document:
        for word in par:
            if word in dictionary:
                dictionary[word] += 1 
            else:
                dictionary[word] = 1
with open('dictionary.pickle', 'wb') as f:
    pickle.dump(dictionary, f)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:06<00:00, 1508.40it/s]


Смотрим какой получился словарь, видим, что всего в коллецкии 44796 различных слов, из них: 14589, 7168, 3423, 2465, 1537 слов, встречающихся во всей коллекции соответственно 1, 2, 3, 4 и 5 раз. 

In [14]:
sorted_dict = sorted(dictionary.items(), key=operator.itemgetter(1))
print(sorted_dict)
k = [0, 0, 0, 0, 0]
for i in dictionary.values():
    if i==1:
        k[0] += 1
    elif i==2:
        k[1] += 1
    elif i== 3:
        k[2] += 1
    elif i==4:
        k[3] += 1
    elif i==5:
        k[4] += 1
print(k)
print(sum(k))
print(len(dictionary))

[('цнииточмаш', 1), ('лыткаринский', 1), ('цниисм', 1), ('ниидвигатель', 1), ('яламов', 1), ('сфд', 1), ('техзадание', 1), ('pzmash', 1), ('бобина', 1), ('амфипатический', 1), ('нуклеозид', 1), ('ek', 1), ('switchbox', 1), ('темников', 1), ('хомячук', 1), ('хивинец', 1), ('опробывание', 1), ('китаев', 1), ('салдыкельский', 1), ('белоглинка', 1), ('гкг', 1), ('гэ', 1), ('добывать', 1), ('щебёночный', 1), ('мониторинговый', 1), ('врезание', 1), ('неметалл', 1), ('тпи', 1), ('роснедра', 1), ('vims', 1), ('эчэ', 1), ('кварц', 1), ('траверс', 1), ('нижнедевицк', 1), ('гердт', 1), ('аижк', 1), ('сбрра', 1), ('videodvor', 1), ('мурсалимкинский', 1), ('суюш', 1), ('тыщука', 1), ('мягкость', 1), ('переустановить', 1), ('длядлить', 1), ('одинарка', 1), ('истра', 1), ('тестироваться', 1), ('мдэ', 1), ('нугуман', 1), ('чичк', 1), ('мирон', 1), ('обуславливаться', 1), ('мелехово', 1), ('чичкевич', 1), ('метрополис', 1), ('шмыкова', 1), ('канун', 1), ('антикоррупцонный', 1), ('жеронско', 1), ('шмыко


[14589, 7168, 3423, 2465, 1537]
29182
44796


Составляю список редких слов

In [51]:
rare_words = []
for word in dictionary.keys():
    if dictionary[word] in [1, 2, 3, 4]:
        rare_words.append(word)
print(len(rare_words))

27645


Пример удаления редких слов

In [ ]:
a = [[[1, 2, 3], [4, 5], [6, 7, 8]], [[9, 10, 11, 12], [13, 14, 15], [16, 17], [18, 19]], [[20, 21], [22, 23, 24], [25, 26]]]
print(a)
rare_words = [22, 2, 5, 15, 7, 10, 26]
i = 0
j = 0
k = 0
while i < len(a):
    while j < len(a[i]):
        while k < len(a[i][j]):
            print('a[', i, j, k, ']', a[i][j][k])
            if a[i][j][k] in rare_words:
                print('delete a[', i, j, k, ']', a[i][j][k])
                del a[i][j][k]
            else:
                k += 1
        k = 0
        j += 1
    j = 0
    i += 1
print(a)

Удаляю слова, количество вхождений которых меньше 5:

In [52]:
with open('points.pickle', 'rb') as f:
    points = pickle.load(f)
i = 0
j = 0
k = 0
print('len(points)', len(points))
while i < len(points):
    while j < len(points[i]):
        while k < len(points[i][j]):
            if points[i][j][k] in rare_words:
                #print('points[', i, j, k, ']', points[i][j][k])
                del points[i][j][k]
            else:
                k += 1
        k = 0
        j += 1
    j = 0
    i += 1

dictionary = dict()
for document in tqdm(points):
    for par in document:
        for word in par:
            if word in dictionary:
                dictionary[word] += 1 
            else:
                dictionary[word] = 1
sorted_dict = sorted(dictionary.items(), key=operator.itemgetter(1))
print(sorted_dict[0])

len(points) 10000


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:06<00:00, 1589.52it/s]


('азовский', 5)


In [17]:
with open('points.pickle', 'wb') as f:
    pickle.dump(points, f)

Удаляю слова, которые являются одной буквой:

In [15]:
t1 = time.time()
i = 0
j = 0
k = 0
print('len(points)', len(points))
while i < len(points):
    while j < len(points[i]):
        while k < len(points[i][j]):
            if points[i][j][k] != '1' and len(points[i][j][k]) == 1 :
                del points[i][j][k]
            else:
                k += 1
        k = 0
        j += 1
    j = 0
    i += 1
t2 = time.time()
print(t2-t1)

len(points) 10000
13.876789331436157


Объединение параграфов малой длины:

In [22]:
with open('points.pickle', 'rb') as f:
    points = pickle.load(f)
with open('is_correct.pickle', 'rb') as f:
    is_correct = pickle.load(f)
i = 0
j = 0
k = 0
while i < len(points):
    while j < len(points[i])-1:
        
        if len(points[i][j]) + len(points[i][j+1]) <= 100:
            points[i][j].extend(points[i][j+1])
            del points[i][j+1]
            del is_correct[i][j+1]
        else:
            j += 1
    j = 0
    i += 1
with open('points.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

In [24]:
with open('points.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

Находим среднюю длину параграфа (для наглядности):

In [23]:
l = []
for i, doc in tqdm(enumerate(points)):
    l.append(0)
    for par in doc:
        l[i] += len(par)
    l[i] = l[i]/len(doc)
print('l =', l[:50])

print(sum(l)/len(l))

10000it [00:00, 138535.15it/s]


l = [74.94117647058823, 85.0, 86.83333333333333, 81.04545454545455, 84.26666666666667, 79.125, 76.38461538461539, 77.13793103448276, 71.86363636363636, 76.56521739130434, 85.77777777777777, 85.29411764705883, 96.08333333333333, 83.33333333333333, 82.9090909090909, 79.47222222222223, 84.25, 81.77777777777777, 86.53846153846153, 177.875, 87.58333333333333, 81.5, 80.6842105263158, 84.45, 302.6, 92.63157894736842, 81.0625, 79.55813953488372, 75.83333333333333, 79.33333333333333, 89.46153846153847, 82.78947368421052, 84.0952380952381, 78.8125, 74.46153846153847, 81.5, 79.70967741935483, 79.4, 80.0, 72.86842105263158, 89.0, 79.6875, 70.38, 89.94117647058823, 80.29411764705883, 90.6923076923077, 83.60526315789474, 83.25, 82.6, 81.71428571428571]
89.0286401895595


Смотрим, сколько документов содержат меньше 5 параграфов:

In [221]:
k = 0
for i, doc in tqdm(enumerate(points)):
    if len(doc)<5:
        print(i, len(doc))
        k += 1
print(k)

0it [00:00, ?it/s]

88 4
142 3
156 4
168 3
200 2
294 4
371 4
376 3
380 3
664 2
718 2
816 3
822 2
907 2
972 3
1051 2
1088 2
1127 3
1171 4
1449 3
1539 4
1554 4
1600 3
1861 4


1862it [00:00, 18029.73it/s]

1922 2
2061 3
2086 4
2203 3
2243 3
2267 4
2450 4
2521 4
2862 2
2870 3
2879 3
2906 4
2942 3
2978 4
3146 3
3323 3
3414 3
3460 3
3521 4
3536 3
3633 3
3636 4
3703 3
3744 3
3751 3
3782 3


3783it [00:00, 18403.71it/s]

...

7945 3
8077 3
8099 3
8106 2
8124 3
8211 4
8280 3
8361 3
8434 2
8566 3
8596 3
8954 3
8986 3
9032 2
9102 3
9159 4
9170 1
9218 4
9279 2
9294 4
9371 4
9471 3
9529 4
9600 4


9601it [00:00, 18775.41it/s]

9635 3
9694 3
9769 4
9909 4


10000it [00:00, 19179.61it/s]


133


Удаляем документы, в которых меньше 5 параграфов:

In [222]:
t1 = time.time()

with open('points.pickle', 'rb') as f:
    points = pickle.load(f)
with open('is_correct.pickle', 'rb') as f:
    is_correct = pickle.load(f)

i = 0
j = 0
k = 0
print('len(points)', len(points))
while i < len(points):
    if len(points[i]) < 5:
        del points[i]
        del is_correct[i]
        
    else:       
        i += 1

t2 = time.time()
print(t2-t1)

len(points) 10000
22.43062472343445


In [223]:
with open('points.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

Заменяем параграфы на другие:

In [83]:
with open('points.pickle', 'rb') as f:
    points = pickle.load(f)
with open('is_correct.pickle', 'rb') as f:
    is_correct = pickle.load(f)

In [225]:
n = 0
m = 0

In [226]:
for i, doc in tqdm(enumerate(points)):
    m += len(doc)
    for j in range(len(doc))[randint(3,10)::7]:
        rand_i = randint(0, len(points)-1)
        points[i][j] = points[rand_i][randint(0, len(points[rand_i])-1)]
        is_correct[i][j] = False
        n += 1

9867it [00:00, 28774.17it/s]


Столько процентов составляют заменённые параграфы (для наглядности):

In [227]:
print(n*100/(m))

11.964376373013726


In [228]:
with open('points_with_anomaly.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct_with_anomaly.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

Снова удаляем те слова, которые встречаются меньше 5 раз во всей коллекции:

In [230]:
t1 = time.time()

with open('points_with_anomaly.pickle', 'rb') as f:
    points = pickle.load(f)

dictionary = dict()
for document in tqdm(points):
    for par in document:
        for word in par:
            if word in dictionary:
                dictionary[word] += 1 
            else:
                dictionary[word] = 1
with open('dictionary.pickle', 'wb') as f:
    pickle.dump(dictionary, f)

rare_words = []
for word in dictionary.keys():
    if dictionary[word] in [1, 2, 3, 4]:
        rare_words.append(word)
print('rare words', len(rare_words)) 
    

i = 0
j = 0
k = 0
print('len(points)', len(points))
while i < len(points):
    while j < len(points[i]):
        while k < len(points[i][j]):
            if points[i][j][k] in rare_words:
                #print('points[', i, j, k, ']', points[i][j][k])
                del points[i][j][k]
            else:
                k += 1
        k = 0
        j += 1
    j = 0
    i += 1

dictionary = dict()
for document in tqdm(points):
    for par in document:
        for word in par:
            if word in dictionary:
                dictionary[word] += 1 
            else:
                dictionary[word] = 1
sorted_dict = sorted(dictionary.items(), key=operator.itemgetter(1))
print(sorted_dict[0])
t2 = time.time()
print(t2-t1)

100%|████████████████████████████████████████████████████████████████████████████| 9867/9867 [00:08<00:00, 1207.95it/s]


rare words 603
len(points) 9867


100%|████████████████████████████████████████████████████████████████████████████| 9867/9867 [00:05<00:00, 1868.99it/s]


('азовский', 5)
217.90949416160583


Количество параграфов во всей коллекции:

In [232]:
m = 0
for doc in points:
    m += len(doc)
print(m)
m = 0
for doc in is_correct:
    m += len(doc)
print(m)

210759
210759


Приводим points в формат листа из листов слов:

In [233]:
sentences = [par for doc in points for par in doc]

In [234]:
print(len(sentences))

210759


Тренируем модель word2vec:

In [235]:
t1 = time.time()

w2vmodel = models.Word2Vec(min_count=4)
w2vmodel.build_vocab(sentences)
w2vmodel.train(sentences, total_examples=w2vmodel.corpus_count, epochs=w2vmodel.iter)
w2vmodel.save('w2v.model')

t2 = time.time()
print(t2 - t1)

C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


48.84268403053284


In [ ]:
print(w2vmodel['покупатель'])
print(w2vmodel['обязанность'])
print(w2vmodel['покупатель'] + w2vmodel['обязанность'])

Модель с rusvectores:

In [208]:
from gensim.models import KeyedVectors
filename = 'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'
filename2 = 'news_upos_cbow_600_2_2018.vec.gz'
wide_w2vmodel = KeyedVectors.load_word2vec_format(filename)

In [209]:
from gensim.models import KeyedVectors
filename2 = 'news_upos_cbow_600_2_2018.vec.gz'
wide_w2vmodel_2 = KeyedVectors.load_word2vec_format(filename2)

In [210]:
wide_w2vmodel.most_similar('покупатель_NOUN')

[('продавец_NOUN', 0.7256076335906982),
 ('потребитель_NOUN', 0.7103174924850464),
 ('клиент_NOUN', 0.7039356231689453),
 ('заказчик_NOUN', 0.6806144118309021),
 ('покупатель_PROPN', 0.6613650321960449),
 ('импортер_NOUN', 0.6521643996238708),
 ('инвестор_NOUN', 0.6466565728187561),
 ('покупательница_NOUN', 0.6445381045341492),
 ('оптовика_NOUN', 0.6391444206237793),
 ('покупщик_NOUN', 0.6388404369354248)]

In [166]:
words = list(wide_w2vmodel_2.wv.vocab)
print(words[:10])

C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


['год_NOUN', 'xx_NUM', 'сообщать_VERB', 'россия_PROPN', 'мочь_VERB', 'человек_NOUN', 'российский_ADJ', 'также_ADV', 'время_NOUN', 'новый_ADJ']


Костыль, после обработки остались пустые параграфы на месте заголовка, поэтому удаляю их тут:

In [237]:
t1 = time.time()
i = 0
j = 0
print('len(points)', len(points))
while i < len(points):
    while j < len(points[i]):
        if points[i][j] == []:
            del points[i][j]
            del is_correct[i][j]
        else:    
            j += 1
    j = 0
    i += 1
t2 = time.time()
print(t2-t1)

len(points) 9867
0.0982353687286377


In [263]:
with open('points_with_anomaly.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct_with_anomaly.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

Каждому параграфу нужно сопоставить векторы X и метку класса Y:

In [239]:
vectors = []
for i, doc in tqdm(enumerate(points)):
    vectors.append([])
    for par in doc:
        vectors[i].append(sum([w2vmodel[word] for word in par]))


0it [00:00, ?it/s]C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
9867it [01:54, 86.34it/s]


In [240]:
m = 0
for doc in points:
    m += len(doc)
print(m)
m = 0
for doc in is_correct:
    m += len(doc)
print(m)

210575
210575


Убедились, что все векторы имеют одинаковую длину 100 (а не 0):

In [241]:
k = 0
for i, doc in enumerate(vectors):
    for j, par in enumerate(doc):
        if type(par) == int:
            print(i,j, par)

In [242]:
k = 0
for i, doc in enumerate(vectors[:1000]):
    for j, par in enumerate(doc):
        if len(par) != 100:
            print(i,j, par)

In [260]:
X = []
Y = []
n = 0
for i, doc in tqdm(enumerate(vectors)):
    
    # для нулевого пераграфа
    X.append([0]*100)
    X[n+0].extend(list(doc[0]))
    X.append(list(doc[0]))

    
    for j, par in enumerate(doc[1:len(doc)-1]):
        par = list(par)
        X[n+j].extend(par)
        X[n+j+1].extend(par)
        X.append(par)
        
    #для последнего параграфа
    X[n+len(doc)-2].extend(doc[-1])
    X[n+len(doc)-1].extend(doc[-1])
    X[n+len(doc)-1].extend([0]*100)
    
    n += len(doc)
    
for doc in tqdm(is_correct):
    for par in doc:
        Y.append(par)

9867it [00:04, 2035.92it/s]
100%|██████████████████████████████████████████████████████████████████████████| 9867/9867 [00:00<00:00, 182222.13it/s]


Проверили, что все элементы множества X - это векторы с размерностью 300, и то, что размерности X и Y совпадают:

In [262]:
print(len(X))
print(len(Y))
for i in X:
    if len(i) != 300:
        print(len(i))

210575
210575


In [264]:
with open('X.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('Y.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

Формируем тренировочное и тестовое множества:

In [272]:
x_train = np.array(X[len(X) * 3 // 10:])
x_test = np.array(X[:len(X) * 3 // 10])
y_train = np.array(Y[len(Y) * 3 // 10:])
y_test = np.array(Y[:len(Y) * 3 // 10])

129805


In [274]:
m_train = 0
for i in y_train:
    if i==True:
        m_train += 1
n_train = len(y_train)
m_test = 0
for i in y_test:
    if i==True:
        m_test += 1
n_test = len(y_test)
print('TRAIN:')
print('total', n_train)
print('True', m_train)
print('False', n_train - m_train)
print('TEST:')
print('total', n_test)
print('True', m_test)
print('False', n_test - m_test)

TRAIN:
total 147403
True 129805
False 17598
TEST:
total 63172
True 55581
False 7591


In [271]:
t1 = time.time()
clf = neighbors.KNeighborsClassifier()
clf.fit(x_train, y_train)
print('classification fitted')
t2 = time.time()
print(t2 - t1)

n = 0
nums = [0, 0]
y = clf.predict(x_test)
for i, j in enumerate(y):
    nums[j] += 1
    if j == y_test[i]:
        n += 1
print('results', n/len(y_test), nums)

print('save clf...')
t1 = time.time()
with open('clf_knn.pickle', 'wb') as f:
    pickle.dump(clf, f)
t2 = time.time()
print(t2 - t1)

classification fitted
5.373748540878296
results 0.9389286392705629 [4795, 58377]
save clf...
1.129068374633789


In [277]:
n = 0
m = 0
for i, j in enumerate(y):
    if j == y_test[i] and j==True:
        n += 1
    elif j == y_test[i]:
        m += 1
    
print('True', n, 'False', m)

True 55050 False 4264
